In [ ]:
import mailbox
import email.utils
import pandas as pd
from datetime import datetime, timezone
import pytz
from ipywidgets import IntProgress, IntText
import docx
import PyPDF2
import gensim
import numpy as np

In [ ]:
filename = "tgthorley.mbox"
filepath = './data/test/'

mbox = mailbox.mbox(filepath + filename)

In [ ]:
def create_email_summary(message):
    emaildate = email.utils.parsedate_to_datetime(message['Date']).astimezone(pytz.utc)
    email_sum = {
            "email_date": emaildate,
            "_id": message['Message-ID'],
            "subject": message['Subject'],
            "text":str()
        }
    return email_sum

In [ ]:
def create_file(part):
    try:
        filename = part.get_filename()
        filepath = "./attachements/"+filename
        fb = open(filepath,'wb')
        fb.write(part.get_payload(decode=True))
        fb.close()
        return filepath
    except:
        print ("File not found", part)
        return None

In [ ]:
def get_text_from_pdf(pdf):
    try:
        filepath = create_file(pdf)
        pdfFileObj = open(filepath,'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        num_pages = pdfReader.numPages
        count = 0
        text = ""
        while count < num_pages:
            pageObj = pdfReader.getPage(count)
            count +=1
            text += pageObj.extractText()
        if text == "":
            print("No Text Found in PDF", filepath)
            #we need to do OCR   
        return text
    except:
        print("File not found", filepath)
        return ""

In [ ]:
def get_text_from_docx(doc):
    filepath = create_file(doc)
    try:
        doc = docx.Document(filepath)
        text = str()
        i=0
        for para in doc.paragraphs:
            text.append("\n" + para[i].text)
            i=i+1
        return text
    except:
        print("File not found", filepath)
        return ""

In [ ]:
emails = []
i=0
progressBar = IntProgress(description='Processing emails '+str(len(mbox)), min=0, max=len(mbox), style = {'description_width': 'initial'})
progressText = IntText(value=0, description='Processed so far: ', style = {'description_width': 'initial'})
display(progressBar, progressText)

for message in mbox:
    email_sum = create_email_summary(message)
    text = str()
    if message.get_content_maintype() == 'multipart':
        for part in message.walk():
            if "text/plain" in part.get_content_type():
                text += part.get_payload()                
            elif "application/msword" in part.get_content_type():
                if "docx" in part.get_filename():
                    text += get_text_from_docx(part) 
            elif "application/pdf" in part.get_content_type():
                text += get_text_from_pdf(part)
        email_sum["text"] = text
        emails.append(email_sum)
    i=i+1
    progressBar.value = i
    progressText.value = i

In [ ]:
df = pd.DataFrame(emails)
df['cleaned_text'] = np.empty((len(df), 0)).tolist()
for index, row in df.iterrows():
    df.loc[index,'cleaned_text'].extend(gensim.utils.simple_preprocess(str(row["text"])))
df